# Load Data Prep

In [1]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame
from itertools import cycle
pd.set_option('display.max_rows',500)

path = os.getcwd()
#print(path)

#this code creates an output directory in the parent director, if one does not exist yet
#Note: sets where all of the output files will be written, since outputs files are large
path = os.getcwd()
parent = os.path.dirname(path)
outputs_dir = parent+'\outputs'
if not os.path.exists(outputs_dir):
    os.makedirs(outputs_dir)
print('output files are written out in parent directory: '+outputs_dir)
print()
load_raw = pd.read_csv('inputs/load_duration_curves_raw_data.csv')
print(load_raw.head(2))
print()
print('number of rows in dataset =', load_raw.shape[0])

output files are written out in parent directory: C:\Users\tgoforth\Documents\IPM temporal resolution project\outputs

     Region  Month  Day    Hour 1    Hour 2    Hour 3    Hour 4    Hour 5  \
0  ERC_REST      1    1   34,807    34,551    34,788    35,531    36,633    
1  ERC_REST      1    2   34,716    34,719    35,076    35,891    37,091    

     Hour 6    Hour 7  ...   Hour 15   Hour 16   Hour 17   Hour 18   Hour 19  \
0   37,780    38,831   ...   38,507    40,084    41,198    40,959    40,549    
1   38,207    38,720   ...   33,211    34,968    37,573    38,213    38,257    

    Hour 20   Hour 21   Hour 22   Hour 23   Hour 24  
0   39,766    38,510    37,012    35,811    35,061   
1   37,911    36,743    35,379    34,598    34,444   

[2 rows x 27 columns]

number of rows in dataset = 27010


In [2]:
#Organizing regional data

#create temporary copy to make changes on
load_org = load_raw.copy()
print('number of rows in dataset (including CN) =',load_org.shape[0])

#Regional IDs
unique_r = pd.Series(load_org['Region'].unique()).dropna()
rl = unique_r.str.split("_",n=1,expand=True)
rl[2] = unique_r
#print(rl)
print('number of regions in dataset (including CN) =',unique_r.shape[0])

#Cleaning up the empty subgroups
#print(rl[rl.isna().any(axis=1)])
rl.loc[rl[0] == 'NENGREST', 1] = 'REST'
rl.loc[rl[0] == 'FRCC', 1] = 'FRCC'

#Cleaning up the misnamed groups
#unique_g = pd.Series(rl[0].unique()).dropna()
#print(unique_g)
rl[0] = rl[0].replace('NENGREST','NENG')
rl[0] = rl[0].replace('WECC','WEC')
unique_g = pd.Series(rl[0].unique()).dropna()
print('number of regional groups in dataset (including CN) =',unique_g.shape[0])
rl.rename(columns={0 : "R_Group", 1: 'R_Subgroup', 2:'Region'},inplace=True)
#print(rl.head())

#Merging Regional Data to DF
load_org = pd.merge(rl,load_org,on='Region',how='right')
print()
print(load_org.head(2))

#Removing Canada
load_org = load_org[load_org['R_Group']!="CN"]
print()
print('number of rows in dataset after removing CN =',load_org.shape[0])
unique_r = pd.Series(load_org['Region'].unique()).dropna()
print('number of regions in dataset (excluding CN) =',unique_r.shape[0])
unique_g = pd.Series(load_org['R_Group'].unique()).dropna()
print('number of regional groups in dataset (excluding CN) =',unique_g.shape[0])

#for testing only, otherwise comment out the lines below
#NOTE: use FRCC for one region, ERC for two regions
#load_org = load_org[load_org['R_Group']=="FRCC"]
#print('number of rows in dataset for testing =',load_org.shape[0])
#Organize temporal data

number of rows in dataset (including CN) = 27010
number of regions in dataset (including CN) = 74
number of regional groups in dataset (including CN) = 10

  R_Group R_Subgroup    Region  Month  Day    Hour 1    Hour 2    Hour 3  \
0     ERC       REST  ERC_REST      1    1   34,807    34,551    34,788    
1     ERC       REST  ERC_REST      1    2   34,716    34,719    35,076    

     Hour 4    Hour 5  ...   Hour 15   Hour 16   Hour 17   Hour 18   Hour 19  \
0   35,531    36,633   ...   38,507    40,084    41,198    40,959    40,549    
1   35,891    37,091   ...   33,211    34,968    37,573    38,213    38,257    

    Hour 20   Hour 21   Hour 22   Hour 23   Hour 24  
0   39,766    38,510    37,012    35,811    35,061   
1   37,911    36,743    35,379    34,598    34,444   

[2 rows x 29 columns]

number of rows in dataset after removing CN = 22995
number of regions in dataset (excluding CN) = 63
number of regional groups in dataset (excluding CN) = 9


In [3]:
#rename hour titles to just the value ('Hour 1' --> 1)
load_org.columns = load_org.columns.str.replace('Hour ', '')
#print(load_org.head(2))

#melt function converts values in wide format to long format
load_dur = pd.melt(load_org,id_vars=['R_Group','R_Subgroup','Region','Month','Day'], \
                   var_name='Hour',value_name='Load')

#print(load_dur.dtypes)

#days are counted 1 to 365, not 1 to 31
unique_d = pd.Series(load_dur['Day'].unique()).dropna()
#print(unique_d.tail(2))

#turn hour values to numeric 
load_dur['Hour'] = pd.to_numeric(load_dur['Hour'],errors='coerce')
unique_h = pd.Series(load_dur['Hour'].unique()).dropna()
#print(unique_h.tail(2))

#turn load values to numeric 
load_dur['Load'] = pd.to_numeric(load_dur['Load'].str.replace(",",""),errors='coerce')
#print(load_dur.head(2))

season_month = pd.read_csv('inputs/season_months.csv')
load_dur = pd.merge(load_dur,season_month, on='Month', how='left')

#organized long format data to new csv file
load_dur = load_dur[['Region','R_Group','R_Subgroup','Season','Month','Day','Hour','Load']]
load_dur.to_csv('../outputs/load_long_format.csv')
print(load_dur.tail(2))
print()
print('number of rows in dataset =',load_dur.shape[0])
print('number of regs in dataset =',load_dur.shape[0]/8760)

         Region R_Group R_Subgroup  Season  Month  Day  Hour  Load
551878  WECC_WY     WEC         WY  winter     12  364    24  1791
551879  WECC_WY     WEC         WY  winter     12  365    24  1834

number of rows in dataset = 551880
number of regs in dataset = 63.0


# Solar Data Prep

In [190]:
solar_raw = pd.read_csv('inputs/solar_generation.csv')
print(solar_raw.head(2))
print()
print('number of rows in dataset =', solar_raw.shape[0])

     Region State  Resource Class  Month  Day  Season  1  2  3  4  ...   15  \
0  ERC_PHDL    TX               6      1    1  Winter  0  0  0  0  ...  516   
1  ERC_PHDL    TX               6      1    2  Winter  0  0  0  0  ...  506   

    16   17   18  19  20  21  22  23  24  
0  548  537  238   0   0   0   0   0   0  
1  547  539  244   0   0   0   0   0   0  

[2 rows x 30 columns]

number of rows in dataset = 58035


In [191]:
#Organizing regional data

#create temporary copy to make changes on
solar_org = solar_raw.copy()
print('number of rows in dataset (including CN) =',solar_org.shape[0])

#Regional IDs
unique_r = pd.Series(solar_org['Region'].unique()).dropna()
rl = unique_r.str.split("_",n=1,expand=True)
rl[2] = unique_r
#print(rl)
print('number of regions in dataset (including CN) =',unique_r.shape[0])

#Cleaning up the empty subgroups
#print(rl[rl.isna().any(axis=1)])
rl.loc[rl[0] == 'NENGREST', 1] = 'REST'
rl.loc[rl[0] == 'FRCC', 1] = 'FRCC'

#Cleaning up the misnamed groups
#unique_g = pd.Series(rl[0].unique()).dropna()
#print(unique_g)
rl[0] = rl[0].replace('NENGREST','NENG')
rl[0] = rl[0].replace('WECC','WEC')
unique_g = pd.Series(rl[0].unique()).dropna()
print('number of regional groups in dataset (including CN) =',unique_g.shape[0])
rl.rename(columns={0 : "R_Group", 1: 'R_Subgroup', 2:'Region'},inplace=True)
#print(rl.head())

#Merging Regional Data to DF
solar_org = pd.merge(rl,solar_org,on='Region',how='right')
print()
print(solar_org.head(2))

#Removing Canada
solar_org = solar_org[solar_org['R_Group']!="CN"]
print()
print('number of rows in dataset after removing CN =',solar_org.shape[0])
unique_r = pd.Series(solar_org['Region'].unique()).dropna()
print('number of regions in dataset (excluding CN) =',unique_r.shape[0])
unique_g = pd.Series(solar_org['R_Group'].unique()).dropna()
print('number of regional groups in dataset (excluding CN) =',unique_g.shape[0])


number of rows in dataset (including CN) = 58035
number of regions in dataset (including CN) = 71
number of regional groups in dataset (including CN) = 10

  R_Group R_Subgroup    Region State  Resource Class  Month  Day  Season  1  \
0     ERC       PHDL  ERC_PHDL    TX               6      1    1  Winter  0   
1     ERC       PHDL  ERC_PHDL    TX               6      1    2  Winter  0   

   2  ...   15   16   17   18  19  20  21  22  23  24  
0  0  ...  516  548  537  238   0   0   0   0   0   0  
1  0  ...  506  547  539  244   0   0   0   0   0   0  

[2 rows x 32 columns]

number of rows in dataset after removing CN = 55115
number of regions in dataset (excluding CN) = 64
number of regional groups in dataset (excluding CN) = 9


In [192]:
#melt function converts values in wide format to long format
solar_dur = pd.melt(solar_org,id_vars=['R_Group','R_Subgroup','Region','State', \
                                       'Season','Month','Day','Resource Class'], \
                    var_name='Hour',value_name='Solar_Gen')
#print(solar_dur)
#print(solar_dur.dtypes)

#turn hour values to numeric 
solar_dur['Hour'] = pd.to_numeric(solar_dur['Hour'],errors='coerce')
unique_h = pd.Series(solar_dur['Hour'].unique()).dropna()
#print(unique_h.tail(2))

#edit columns
solar_dur = solar_dur.drop(columns={'Season'})
solar_dur = solar_dur.rename(columns={'Resource Class':'TRG'})
#print(solar_dur)

#change generation value to capacity factor
solar_dur['Solar_Gen']=solar_dur['Solar_Gen']/1000

#create new seasons column
season_month = pd.read_csv('inputs/season_months.csv')
solar_dur = pd.merge(solar_dur,season_month, on='Month', how='left')
solar_dur = solar_dur.sort_values(['Region','Month','Day'])
print(solar_dur.head())
#print(solar_dur.shape)

       R_Group R_Subgroup    Region State  Month  Day  TRG  Hour  Solar_Gen  \
0          ERC       PHDL  ERC_PHDL    TX      1    1    6     1        0.0   
365        ERC       PHDL  ERC_PHDL    TX      1    1    7     1        0.0   
55115      ERC       PHDL  ERC_PHDL    TX      1    1    6     2        0.0   
55480      ERC       PHDL  ERC_PHDL    TX      1    1    7     2        0.0   
110230     ERC       PHDL  ERC_PHDL    TX      1    1    6     3        0.0   

        Season  
0       winter  
365     winter  
55115   winter  
55480   winter  
110230  winter  


In [193]:
#filter by TRG because too big otherwise 
solar3 = solar_dur.loc[solar_dur['TRG'] == 3].reset_index(drop=True)
solar3 = solar3.rename(columns={'Solar_Gen':'TRG3'}).drop(columns={'TRG'})

solar4 = solar_dur.loc[solar_dur['TRG'] == 4].reset_index(drop=True)
solar4 = solar4.rename(columns={'Solar_Gen':'TRG4'}).drop(columns={'TRG'})

solar5 = solar_dur.loc[solar_dur['TRG'] == 5].reset_index(drop=True)
solar5 = solar5.rename(columns={'Solar_Gen':'TRG5'}).drop(columns={'TRG'})

solar6 = solar_dur.loc[solar_dur['TRG'] == 6].reset_index(drop=True)
solar6 = solar6.rename(columns={'Solar_Gen':'TRG6'}).drop(columns={'TRG'})

solar7 = solar_dur.loc[solar_dur['TRG'] == 7].reset_index(drop=True)
solar7 = solar7.rename(columns={'Solar_Gen':'TRG7'}).drop(columns={'TRG'})

solar8 = solar_dur.loc[solar_dur['TRG'] == 8].reset_index(drop=True)
solar8 = solar8.rename(columns={'Solar_Gen':'TRG8'}).drop(columns={'TRG'})

In [194]:
#create DF that only has the labels, easier to merge onto
solar_labels = solar_dur[['Region','R_Group','R_Subgroup','State','Month','Day','Hour','Season']].copy()
solar_labels = solar_labels.drop_duplicates(['Region','R_Group','R_Subgroup','State','Month','Day','Hour'])
#print(solar_labels)

solar_dur2 = pd.merge(solar_labels,solar3,on=['R_Group','R_Subgroup','Region','State','Month','Day','Hour','Season'],how='left')
solar_dur2 = pd.merge(solar_dur2,solar4,on=['R_Group','R_Subgroup','Region','State','Month','Day','Hour','Season'],how='left')
solar_dur2 = pd.merge(solar_dur2,solar5,on=['R_Group','R_Subgroup','Region','State','Month','Day','Hour','Season'],how='left')
solar_dur2 = pd.merge(solar_dur2,solar6,on=['R_Group','R_Subgroup','Region','State','Month','Day','Hour','Season'],how='left')
solar_dur2 = pd.merge(solar_dur2,solar7,on=['R_Group','R_Subgroup','Region','State','Month','Day','Hour','Season'],how='left')
solar_dur2 = pd.merge(solar_dur2,solar8,on=['R_Group','R_Subgroup','Region','State','Month','Day','Hour','Season'],how='left')
print(solar_dur2.head())
print()
print('number of rows in dataset =',solar_dur2.shape[0])
print('number of regs in dataset =',solar_dur2.shape[0]/8760)

     Region R_Group R_Subgroup State  Month  Day  Hour  Season  TRG3  TRG4  \
0  ERC_PHDL     ERC       PHDL    TX      1    1     1  winter   NaN   NaN   
1  ERC_PHDL     ERC       PHDL    TX      1    1     2  winter   NaN   NaN   
2  ERC_PHDL     ERC       PHDL    TX      1    1     3  winter   NaN   NaN   
3  ERC_PHDL     ERC       PHDL    TX      1    1     4  winter   NaN   NaN   
4  ERC_PHDL     ERC       PHDL    TX      1    1     5  winter   NaN   NaN   

   TRG5  TRG6  TRG7  TRG8  
0   NaN   0.0   0.0   NaN  
1   NaN   0.0   0.0   NaN  
2   NaN   0.0   0.0   NaN  
3   NaN   0.0   0.0   NaN  
4   NaN   0.0   0.0   NaN  

number of rows in dataset = 683280
number of regs in dataset = 78.0


In [195]:
#matches the month and day-of-month to the day-of-year (e.g.365) value
days = pd.read_csv('inputs/days_365.csv')
#print(days.head())
solar_dur2.rename(columns={'Day':'DayofMo'}, inplace=True)
#print(solar_dur2.head())
solar_dur3 = pd.merge(days,solar_dur2,on=['Month','DayofMo'],how='right')
#print(solar_dur3.head())

#organized long format data to new csv file
solar_dur3 = solar_dur3[['Region','R_Group','R_Subgroup','State','Season','Month','Day','Hour',
                       'TRG3','TRG4','TRG5','TRG6','TRG7','TRG8']]
solar_dur3.to_csv('../outputs/solar_long_format.csv')
print(solar_dur3.head())
print('number of rows in dataset =',solar_dur3.shape[0])
print('number of regs in dataset =',solar_dur2.shape[0]/8760)

     Region R_Group R_Subgroup State  Season  Month  Day  Hour  TRG3  TRG4  \
0  ERC_PHDL     ERC       PHDL    TX  winter      1    1     1   NaN   NaN   
1  ERC_PHDL     ERC       PHDL    TX  winter      1    1     2   NaN   NaN   
2  ERC_PHDL     ERC       PHDL    TX  winter      1    1     3   NaN   NaN   
3  ERC_PHDL     ERC       PHDL    TX  winter      1    1     4   NaN   NaN   
4  ERC_PHDL     ERC       PHDL    TX  winter      1    1     5   NaN   NaN   

   TRG5  TRG6  TRG7  TRG8  
0   NaN   0.0   0.0   NaN  
1   NaN   0.0   0.0   NaN  
2   NaN   0.0   0.0   NaN  
3   NaN   0.0   0.0   NaN  
4   NaN   0.0   0.0   NaN  
number of rows in dataset = 683280
number of regs in dataset = 78.0


# Wind Data Prep

In [196]:
wind_raw = pd.read_csv('inputs/onshore_wind_gen.csv')
print(wind_raw.head(2))
print()
print('number of rows in dataset =', wind_raw.shape[0])

     Region State  TRG  Month  Day    1    2    3    4    5  ...   15   16  \
0  ERC_PHDL    TX    1      1    1  758  784  720  680  647  ...  250  265   
1  ERC_PHDL    TX    1      1    2  720  752  745  737  718  ...  384  401   

    17   18   19   20   21   22   23   24  
0  284  278  318  428  543  666  689  700  
1  368  335  270  386  497  611  595  681  

[2 rows x 29 columns]

number of rows in dataset = 247105


In [197]:
#Organizing regional data

#create temporary copy to make changes on
wind_org = wind_raw.copy()
print('number of rows in dataset (including CN) =',wind_org.shape[0])

#Regional IDs
unique_r = pd.Series(wind_org['Region'].unique()).dropna()
rl = unique_r.str.split("_",n=1,expand=True)
rl[2] = unique_r
#print(rl)
print('number of regions in dataset (including CN) =',unique_r.shape[0])

#Cleaning up the empty subgroups
#print(rl[rl.isna().any(axis=1)])
rl.loc[rl[0] == 'NENGREST', 1] = 'REST'
rl.loc[rl[0] == 'FRCC', 1] = 'FRCC'

#Cleaning up the misnamed groups
#unique_g = pd.Series(rl[0].unique()).dropna()
#print(unique_g)
rl[0] = rl[0].replace('NENGREST','NENG')
rl[0] = rl[0].replace('WECC','WEC')
unique_g = pd.Series(rl[0].unique()).dropna()
print('number of regional groups in dataset (including CN) =',unique_g.shape[0])
rl.rename(columns={0 : "R_Group", 1: 'R_Subgroup', 2:'Region'},inplace=True)
#print(rl.head())

#Merging Regional Data to DF
wind_org = pd.merge(rl,wind_org,on='Region',how='right')
print()
print(wind_org.head(2))

#Removing Canada
wind_org = wind_org[wind_org['R_Group']!="CN"]
print()
print('number of rows in dataset after removing CN =',wind_org.shape[0])
unique_r = pd.Series(wind_org['Region'].unique()).dropna()
print('number of regions in dataset (excluding CN) =',unique_r.shape[0])
unique_g = pd.Series(wind_org['R_Group'].unique()).dropna()
print('number of regional groups in dataset (excluding CN) =',unique_g.shape[0])


number of rows in dataset (including CN) = 247105
number of regions in dataset (including CN) = 69
number of regional groups in dataset (including CN) = 10

  R_Group R_Subgroup    Region State  TRG  Month  Day    1    2    3  ...  \
0     ERC       PHDL  ERC_PHDL    TX    1      1    1  758  784  720  ...   
1     ERC       PHDL  ERC_PHDL    TX    1      1    2  720  752  745  ...   

    15   16   17   18   19   20   21   22   23   24  
0  250  265  284  278  318  428  543  666  689  700  
1  384  401  368  335  270  386  497  611  595  681  

[2 rows x 31 columns]

number of rows in dataset after removing CN = 226665
number of regions in dataset (excluding CN) = 63
number of regional groups in dataset (excluding CN) = 9


In [198]:
#locate each TRG in the DF 
wind1 = wind_org.loc[wind_org['TRG'] == 1].reset_index(drop=True)
wind2 = wind_org.loc[wind_org['TRG'] == 2].reset_index(drop=True)
wind3 = wind_org.loc[wind_org['TRG'] == 3].reset_index(drop=True)
wind4 = wind_org.loc[wind_org['TRG'] == 4].reset_index(drop=True)
wind5 = wind_org.loc[wind_org['TRG'] == 5].reset_index(drop=True)
wind6 = wind_org.loc[wind_org['TRG'] == 6].reset_index(drop=True)
print(wind6.head())

  R_Group R_Subgroup    Region State  TRG  Month  Day    1    2    3  ...  \
0     ERC       PHDL  ERC_PHDL    TX    6      1    1  470  451  469  ...   
1     ERC       PHDL  ERC_PHDL    TX    6      1    2  630  614  592  ...   
2     ERC       PHDL  ERC_PHDL    TX    6      1    3  674  685  670  ...   
3     ERC       PHDL  ERC_PHDL    TX    6      1    4  633  589  559  ...   
4     ERC       PHDL  ERC_PHDL    TX    6      1    5  467  475  481  ...   

    15   16   17   18   19   20   21   22   23   24  
0  410  469  490  475  474  479  484  525  585  611  
1  278  316  340  364  320  354  384  489  579  631  
2  472  464  448  422  439  512  560  630  661  646  
3  188  208  216  196  200  233  284  403  430  444  
4  263  286  289  298  270  281  310  386  377  394  

[5 rows x 31 columns]


### Melt each TRG invidually, then combine. Otherwise the file is too large. 
### TRG 1 

In [199]:
#melt function converts values in wide format to long format
wind_dur1 = pd.melt(wind1,id_vars=['R_Group','R_Subgroup','Region','State','Month','Day','TRG'],
                    var_name='Hour',value_name='Wind_Gen')
wind_dur1['Wind_Gen']=wind_dur1['Wind_Gen']/1000
#print(wind_dur1.head())
#print(wind_dur1.dtypes)

unique_d = pd.Series(wind_dur1['Day'].unique()).dropna()
#print(unique_d.tail(2))

#turn hour values to numeric 
wind_dur1['Hour'] = pd.to_numeric(wind_dur1['Hour'],errors='coerce')
unique_h = pd.Series(wind_dur1['Hour'].unique()).dropna()
#print(unique_h.tail(2))

#create new seasons column
season_month = pd.read_csv('inputs/season_months.csv')
wind_dur1 = pd.merge(wind_dur1,season_month, on='Month', how='left')

#edit columns 
wind_dur1 = wind_dur1.rename(columns={'Wind_Gen':'TRG1'}).drop(columns={'TRG'})
wind_dur1 = wind_dur1[['Region','R_Group','R_Subgroup','State','Season','Month','Day','Hour','TRG1']]
wind_dur1 = wind_dur1.sort_values(['Region','Month','Day'])
print(wind_dur1.head())
print()
print('number of rows in dataset =',wind_dur1.shape[0])

         Region R_Group R_Subgroup State  Season  Month  Day  Hour   TRG1
0      ERC_PHDL     ERC       PHDL    TX  winter      1    1     1  0.758
18980  ERC_PHDL     ERC       PHDL    TX  winter      1    1     2  0.784
37960  ERC_PHDL     ERC       PHDL    TX  winter      1    1     3  0.720
56940  ERC_PHDL     ERC       PHDL    TX  winter      1    1     4  0.680
75920  ERC_PHDL     ERC       PHDL    TX  winter      1    1     5  0.647

number of rows in dataset = 455520


### TRG 2

In [200]:
#melt function converts values in wide format to long format
wind_dur2 = pd.melt(wind2,id_vars=['R_Group','R_Subgroup','Region','State','Month','Day','TRG'],
                    var_name='Hour',value_name='Wind_Gen')
wind_dur2['Wind_Gen']=wind_dur2['Wind_Gen']/1000
#print(wind_dur2)
#print(wind_dur2.dtypes)

#
unique_d = pd.Series(wind_dur2['Day'].unique()).dropna()
#print(unique_d.tail(2))

#turn hour values to numeric 
wind_dur2['Hour'] = pd.to_numeric(wind_dur2['Hour'],errors='coerce')
unique_h = pd.Series(wind_dur2['Hour'].unique()).dropna()
#print(unique_h.tail(2))

#create new seasons column
season_month = pd.read_csv('inputs/season_months.csv')
wind_dur2 = pd.merge(wind_dur2,season_month, on='Month', how='left')

#edit columns 
wind_dur2 = wind_dur2.rename(columns={'Wind_Gen':'TRG2'}).drop(columns={'TRG'})
print(wind_dur2.tail(2))
print()
print('number of rows in dataset =',wind_dur2.shape[0])

       R_Group R_Subgroup   Region State  Month  Day  Hour   TRG2  Season
437998     WEC         WY  WECC_WY    WY     12   30    24  0.615  winter
437999     WEC         WY  WECC_WY    WY     12   31    24  0.864  winter

number of rows in dataset = 438000


### TRG 3

In [201]:
#melt function converts values in wide format to long format
wind_dur3 = pd.melt(wind3,id_vars=['R_Group','R_Subgroup','Region','State','Month','Day','TRG'],
                    var_name='Hour',value_name='Wind_Gen')
wind_dur3['Wind_Gen']=wind_dur3['Wind_Gen']/1000
#print(wind_dur3)
#print(wind_dur3.dtypes)

unique_d = pd.Series(wind_dur3['Day'].unique()).dropna()
#print(unique_d.tail(2))

#turn hour values to numeric 
wind_dur3['Hour'] = pd.to_numeric(wind_dur3['Hour'],errors='coerce')
unique_h = pd.Series(wind_dur3['Hour'].unique()).dropna()
#print(unique_h.tail(2))

#create new seasons column
season_month = pd.read_csv('inputs/season_months.csv')
wind_dur3 = pd.merge(wind_dur3,season_month, on='Month', how='left')

#edit columns 
wind_dur3 = wind_dur3.rename(columns={'Wind_Gen':'TRG3'}).drop(columns={'TRG'})
print(wind_dur3.tail(2))
print()
print('number of rows in dataset =',wind_dur3.shape[0])

       R_Group R_Subgroup   Region State  Month  Day  Hour   TRG3  Season
508078     WEC         WY  WECC_WY    WY     12   30    24  0.657  winter
508079     WEC         WY  WECC_WY    WY     12   31    24  0.883  winter

number of rows in dataset = 508080


### TRG 4

In [202]:
#melt function converts values in wide format to long format
wind_dur4 = pd.melt(wind4,id_vars=['R_Group','R_Subgroup','Region','State','Month','Day','TRG'],
                    var_name='Hour',value_name='Wind_Gen')
wind_dur4['Wind_Gen']=wind_dur4['Wind_Gen']/1000
#print(wind_dur4)
#print(wind_dur4.dtypes)

unique_d = pd.Series(wind_dur4['Day'].unique()).dropna()
#print(unique_d.tail(2))

#turn hour values to numeric 
wind_dur4['Hour'] = pd.to_numeric(wind_dur4['Hour'],errors='coerce')
unique_h = pd.Series(wind_dur4['Hour'].unique()).dropna()
#print(unique_h.tail(2))

#create new seasons column
season_month = pd.read_csv('inputs/season_months.csv')
wind_dur4 = pd.merge(wind_dur4,season_month, on='Month', how='left')

#edit columns 
wind_dur4 = wind_dur4.rename(columns={'Wind_Gen':'TRG4'}).drop(columns={'TRG'})
print(wind_dur4.tail(2))
print()
print('number of rows in dataset =',wind_dur4.shape[0])

       R_Group R_Subgroup   Region State  Month  Day  Hour   TRG4  Season
578158     WEC         WY  WECC_WY    WY     12   30    24  0.672  winter
578159     WEC         WY  WECC_WY    WY     12   31    24  0.880  winter

number of rows in dataset = 578160


### TRG 5

In [203]:
#melt function converts values in wide format to long format
wind_dur5 = pd.melt(wind5,id_vars=['R_Group','R_Subgroup','Region','State','Month','Day','TRG'],
                    var_name='Hour',value_name='Wind_Gen')
wind_dur5['Wind_Gen']=wind_dur5['Wind_Gen']/1000
#print(wind_dur5)
#print(wind_dur5.dtypes)

unique_d = pd.Series(wind_dur5['Day'].unique()).dropna()
#print(unique_d.tail(2))

#turn hour values to numeric 
wind_dur5['Hour'] = pd.to_numeric(wind_dur5['Hour'],errors='coerce')
unique_h = pd.Series(wind_dur5['Hour'].unique()).dropna()
#print(unique_h.tail(2))

#create new seasons column
season_month = pd.read_csv('inputs/season_months.csv')
wind_dur5 = pd.merge(wind_dur5,season_month, on='Month', how='left')

#edit columns 
wind_dur5 = wind_dur5.rename(columns={'Wind_Gen':'TRG5'}).drop(columns={'TRG'})
print(wind_dur5.tail(2))
print()
print('number of rows in dataset =',wind_dur5.shape[0])

       R_Group R_Subgroup   Region State  Month  Day  Hour   TRG5  Season
727078     WEC         WY  WECC_WY    WY     12   30    24  0.637  winter
727079     WEC         WY  WECC_WY    WY     12   31    24  0.792  winter

number of rows in dataset = 727080


### TRG 6

In [204]:
#melt function converts values in wide format to long format
wind_dur6 = pd.melt(wind6,id_vars=['R_Group','R_Subgroup','Region','State','Month','Day','TRG'],
                    var_name='Hour',value_name='Wind_Gen')
wind_dur6['Wind_Gen']=wind_dur6['Wind_Gen']/1000
#print(wind_dur6)
#print(wind_dur6.dtypes)

unique_d = pd.Series(wind_dur6['Day'].unique()).dropna()
#print(unique_d.tail(2))

#turn hour values to numeric 
wind_dur6['Hour'] = pd.to_numeric(wind_dur6['Hour'],errors='coerce')
unique_h = pd.Series(wind_dur6['Hour'].unique()).dropna()
#print(unique_h.tail(2))

#create new seasons column
season_month = pd.read_csv('inputs/season_months.csv')
wind_dur6 = pd.merge(wind_dur6,season_month, on='Month', how='left')

#edit columns 
wind_dur6 = wind_dur6.rename(columns={'Wind_Gen':'TRG6'}).drop(columns={'TRG'})
print(wind_dur6.tail(2))
print()
print('number of rows in dataset =',wind_dur6.shape[0])

       R_Group R_Subgroup   Region State  Month  Day  Hour   TRG6  Season
832198     WEC         WY  WECC_WY    WY     12   30    24  0.505  winter
832199     WEC         WY  WECC_WY    WY     12   31    24  0.605  winter

number of rows in dataset = 832200


In [205]:
#merge into one DF 
wind_dur = pd.merge(wind_dur1,wind_dur2,on=['Region','R_Group','R_Subgroup','State',
                                            'Season','Month','Day','Hour'],how='outer')
wind_dur = pd.merge(wind_dur,wind_dur3,on=['Region','R_Group','R_Subgroup','State',
                                           'Season','Month','Day','Hour'], how='outer')
wind_dur = pd.merge(wind_dur,wind_dur4,on=['Region','R_Group','R_Subgroup','State',
                                           'Season','Month','Day','Hour'], how='outer')
wind_dur = pd.merge(wind_dur,wind_dur5,on=['Region','R_Group','R_Subgroup','State',
                                           'Season','Month','Day','Hour'], how='outer')
wind_dur = pd.merge(wind_dur,wind_dur6,on=['Region','R_Group','R_Subgroup','State',
                                           'Season','Month','Day','Hour'], how='outer')
print(wind_dur.head())

     Region R_Group R_Subgroup State  Season  Month  Day  Hour   TRG1   TRG2  \
0  ERC_PHDL     ERC       PHDL    TX  winter      1    1     1  0.758  0.788   
1  ERC_PHDL     ERC       PHDL    TX  winter      1    1     2  0.784  0.785   
2  ERC_PHDL     ERC       PHDL    TX  winter      1    1     3  0.720  0.719   
3  ERC_PHDL     ERC       PHDL    TX  winter      1    1     4  0.680  0.695   
4  ERC_PHDL     ERC       PHDL    TX  winter      1    1     5  0.647  0.728   

    TRG3   TRG4   TRG5   TRG6  
0  0.713  0.653  0.573  0.470  
1  0.718  0.687  0.555  0.451  
2  0.642  0.605  0.567  0.469  
3  0.585  0.579  0.572  0.453  
4  0.583  0.538  0.599  0.487  


In [206]:
#matches the month and day-of-month to the day-of-year (e.g.365) value
#print(days.head())
wind_dur.rename(columns={'Day':'DayofMo'}, inplace=True)
#print(wind_dur.head())
wind_dur_fin = pd.merge(days,wind_dur,on=['Month','DayofMo'],how='right')
#print(wind_dur_fin.head())

#organized long format data to new csv file
wind_dur_fin = wind_dur_fin[['Region','R_Group','R_Subgroup','State','Season','Month','Day','Hour',\
                             'TRG1','TRG2','TRG3','TRG4','TRG5','TRG6']]
wind_dur_fin.to_csv('../outputs/wind_long_format.csv')
print(wind_dur_fin.head())
print('number of rows in dataset =',wind_dur_fin.shape[0])
print('number of regs in dataset =',solar_dur2.shape[0]/8760)


     Region R_Group R_Subgroup State  Season  Month  Day  Hour   TRG1   TRG2  \
0  ERC_PHDL     ERC       PHDL    TX  winter      1    1     1  0.758  0.788   
1  ERC_PHDL     ERC       PHDL    TX  winter      1    1     2  0.784  0.785   
2  ERC_PHDL     ERC       PHDL    TX  winter      1    1     3  0.720  0.719   
3  ERC_PHDL     ERC       PHDL    TX  winter      1    1     4  0.680  0.695   
4  ERC_PHDL     ERC       PHDL    TX  winter      1    1     5  0.647  0.728   

    TRG3   TRG4   TRG5   TRG6  
0  0.713  0.653  0.573  0.470  
1  0.718  0.687  0.555  0.451  
2  0.642  0.605  0.567  0.469  
3  0.585  0.579  0.572  0.453  
4  0.583  0.538  0.599  0.487  
number of rows in dataset = 867240
number of regs in dataset = 78.0
